In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
# Load the dataset
dataset_path = 'IMDB_Dataset.csv'
data = pd.read_csv(dataset_path)


In [22]:
# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review'])
X = tokenizer.texts_to_sequences(data['review'])
X = pad_sequences(X, maxlen=200)

In [23]:
# Convert sentiment to binary labels
y = np.where(data['sentiment']=='positive', 1, 0)

In [24]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
# Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=200))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [26]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [28]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[early_stopping])

Epoch 1/10
625/625 [==============================] - 69s 107ms/step - loss: 0.4029 - accuracy: 0.8170 - val_loss: 0.2973 - val_accuracy: 0.8806
Epoch 2/10
625/625 [==============================] - 26s 41ms/step - loss: 0.1948 - accuracy: 0.9276 - val_loss: 0.3158 - val_accuracy: 0.8671
Epoch 3/10
625/625 [==============================] - 17s 27ms/step - loss: 0.1034 - accuracy: 0.9649 - val_loss: 0.3050 - val_accuracy: 0.8815
Epoch 4/10
625/625 [==============================] - 12s 19ms/step - loss: 0.0569 - accuracy: 0.9817 - val_loss: 0.4322 - val_accuracy: 0.8789


In [29]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

313/313 [==============================] - 2s 5ms/step - loss: 0.4322 - accuracy: 0.8789
Test Loss: 0.43216854333877563, Test Accuracy: 0.8788999915122986
